In [45]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.serialize import model_from_json
import datetime
import itertools
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


models_list = ['non_dom_d', 'non_dom_m']
models_params = dict(zip(models_list, [('D', 'Non-domestic'), ('M', 'Non-domestic')]))
gdp_dict = {"ds" : [2015_1,2015_2,2015_3,2015_4,2015_5,2015_6,2015_7,2015_8,2015_9,2015_10,2015_11,2015_12,2016_1,2016_2,2016_3,2016_4,2016_5,2016_6,2016_7,2016_8,2016_9,2016_10,2016_11,2016_12,2017_1,2017_2,2017_3,2017_4,2017_5,2017_6,2017_7,2017_8,2017_9,2017_10,2017_11,2017_12,2018_1,2018_2,2018_3,2018_4,2018_5,2018_6,2018_7,2018_8,2018_9,2018_10,2018_11,2018_12,2019_1,2019_2,2019_3,2019_4,2019_5,2019_6,2019_7,2019_8,2019_9,2019_10,2019_11,2019_12,2020_1,2020_2,2020_3,2020_4,2020_5,2020_6,2020_7,2020_8,2020_9,2020_10,2020_11,2020_12,2021_1,2021_2,2021_3,2021_4,2021_5,2021_6,2021_7,2021_8,2021_9,2021_10,2021_11,2021_12,2022_1,2022_2,2022_3,2022_4,2022_5,2022_6,2022_7,2022_8,2022_9,2022_10,2022_11,2022_12,2023_1,2023_2,2023_3,2023_4,2023_5,2023_6,2023_7,2023_8,2023_9,2023_10,2023_11,2023_12,2024_1,2024_2,2024_3,2024_4,2024_5,2024_6,2024_7,2024_8,2024_9,2024_10,2024_11,2024_12,2025_1,2025_2,2025_3,2025_4,2025_5,2025_6,2025_7,2025_8,2025_9,2025_10,2025_11,2025_12,2026_1,2026_2,2026_3,2026_4,2026_5,2026_6,2026_7,2026_8,2026_9,2026_10,2026_11,2026_12,2027_1,2027_2,2027_3,2027_4,2027_5,2027_6,2027_7,2027_8,2027_9,2027_10,2027_11,2027_12,2028_1,2028_2,2028_3,2028_4,2028_5,2028_6,2028_7,2028_8,2028_9,2028_10,2028_11,2028_12,2029_1,2029_2,2029_3,2029_4,2029_5,2029_6,2029_7,2029_8,2029_9,2029_10,2029_11,2029_12,2030_1,2030_2,2030_3,2030_4,2030_5,2030_6,2030_7,2030_8,2030_9,2030_10,2030_11,2030_12,2031_1,2031_2,2031_3], "gdp" : [91.687,91.961,92.0195,92.5513,92.3066,92.2139,92.6526,92.5197,92.6516,93.0295,92.8692,93.2685,93.2419,93.5345,93.7219,94.2234,94.0166,94.1336,94.2428,94.5378,95.0462,94.737,95.2482,95.9471,96.0813,96.1087,96.2108,96.613,96.7602,96.7748,96.9762,97.1614,97.5765,97.6088,97.8274,98.0528,97.8321,97.8775,98.0007,97.769,98.0835,98.4296,98.2858,98.4504,98.5964,98.495,98.7516,98.4909,98.8571,99.5299,99.5556,99.3054,99.5938,100.0858,100.5729,100.2885,100.6168,100.4694,100.3002,100.8246,100.8145,100.444,92.7769,74.5691,76.1555,83.5776,89.126,91.8198,93.026,93.5677,91.6681,92.8976,90.3167,91.2726,93.3356,96.7661,98.2848,99.4212,98.8991,99.7767,100.3606,100.7081,101.4433,100.9219,101.4331,101.966,102.1056,101.8883,102.5782,101.5609,101.9985,102.0639,101.4602,102.1466,102.205,101.8905,102.4213,102.5735,102.2569,102.4903,102.2648,103.0142,102.3513,102.5317,102.666627583333,102.6837162,102.70089381785,102.752095055675,102.8033865386,102.854945158577,102.906223198259,102.957591618408,103.009227576315,103.060582533056,103.112028005836,103.163741417679,103.215173406855,103.266696047844,103.318487029806,103.455844728899,103.593444528767,103.731760977925,103.869668107815,104.007818306882,104.146688021837,104.285146780246,104.423849580109,104.563274773924,104.702287367367,104.84154497843,104.98152787302,105.147265637552,105.313295524342,105.480190130416,105.646715149331,105.813533678082,105.981221033536,106.14853704629,106.316147963053,106.484631833445,106.65274259726,106.821149665878,106.990433834654,107.159343124597,107.328550126791,107.498638395369,107.668350004439,107.838360739893,108.009256927747,108.17977466696,108.350592953407,108.522300898154,108.693628596628,108.865258269936,109.03778182742,109.218983411675,109.40050437405,109.582970736557,109.765078328733,109.94750689592,110.13088559024,110.313903720377,110.4972444304,110.681540018191,110.865473238978,111.049730652552,111.234947718282,111.419800605173,111.604979305814,111.791122456873,111.976899608199,112.163004202343,112.350078069157,112.53678410624,112.723819223355,112.911828459503,113.099468026771,113.287438319472,113.476387601801,113.664965366905,113.853875511069,114.04376953981,114.23329019374,114.423144888625,114.613988387509,114.804456644708,114.995260613068,115.187058329446,115.378478927932,115.570236916133,115.762993621094,115.955371322572,116.148088100714,116.341808589199,116.535148179184] }

daily_outputs_path = "../../outputs/daily/Non-domestic_Daily_reduced_forecast.parquet"
monthly_outputs_path = "../../outputs/monthly/Non-domestic_Monthly_reduced_forecast.parquet"


In [48]:
daily_fc = pd.read_parquet(daily_outputs_path)
monthly_fc = pd.read_parquet(monthly_outputs_path)

In [49]:
### Getting the linear models for the regressors, which are the way Prophet takes them in.

## GDP
gdp_x = monthly_fc.dropna(subset=['y'])['nominal_gdp'].to_numpy().reshape(-1,1)
gdp_y = monthly_fc.dropna(subset=['y'])['gdp'].to_numpy().reshape(-1,1)
gdp_regr = LinearRegression()
gdp_regressor = gdp_regr.fit(gdp_x,gdp_y)

## Weather
temperature_x = daily_fc.dropna(subset=['y'])['nominal_temperature'].to_numpy().reshape(-1,1)
temperature_y = daily_fc.dropna(subset=['y'])['temperature'].to_numpy().reshape(-1,1)
temperature_regr = LinearRegression()
temperature_regressors = temperature_regr.fit(temperature_x,temperature_y)

cdd_x = daily_fc.dropna(subset=['y'])['nominal_cdd'].to_numpy().reshape(-1,1)
cdd_y = daily_fc.dropna(subset=['y'])['cdd'].to_numpy().reshape(-1,1)
cdd_regr = LinearRegression()
cdd_regressors = cdd_regr.fit(cdd_x,cdd_y)

hdd_x = daily_fc.dropna(subset=['y'])['nominal_hdd'].to_numpy().reshape(-1,1)
hdd_y = daily_fc.dropna(subset=['y'])['hdd'].to_numpy().reshape(-1,1)
hdd_regr = LinearRegression()
hdd_regressors = hdd_regr.fit(hdd_x,hdd_y)

In [50]:
### Mini-models for approximating hdd and cdd from the temperature.

# Extrapolating temperature to HDD:
def get_hdd(self, temperature : np.array):
  """
  temperature: numpy array 2D
  """
  X = daily_fc.dropna(subset=['y'])['nominal_temperature'].to_numpy().reshape(-1,1)
  y = daily_fc.dropna(subset=['y'])['nominal_hdd'].to_numpy().reshape(-1,1)
  degree = 2 
  polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())
  polyreg.fit(X, y)
  prediction = polyreg.predict(temperature.reshape(-1,1)).flatten()
  prediction[prediction < 0] = 0 
  prediction[temperature.flatten() > 18] = 0
  return prediction

# Extrapolating temperature to CDD:
def get_cdd(self, temperature : np.array):
  """
  temperature: numpy array 2D
  """
  X = daily_fc.dropna(subset=['y'])['nominal_temperature'].to_numpy().reshape(-1,1)
  y = daily_fc.dropna(subset=['y'])['nominal_cdd'].to_numpy().reshape(-1,1)
  degree = 3 
  polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())
  polyreg.fit(X, y) 
  prediction = polyreg.predict(temperature.reshape(-1,1)).flatten()
  prediction[prediction < 0] = 0 
  prediction[temperature.flatten() < 10] = 0 
  return prediction


In [51]:
def generate_macro_scenarios(monthly_fc, gdp_scenarios : itertools.product):
      """
      Paramters: List of GDP figures.
      """
      monthly_cols = ['ds','nominal_gdp', 'trend', 'multiplicative_terms', 'gdp']
      macro_scenarios_names = []
      counter = 0

      for gdp_dict in gdp_scenarios:

            gdp = pd.DataFrame(gdp_dict)
            gdp['ds'] = gdp['ds'].apply(lambda x: datetime.date(int(str(x)[:4]),int(str(x)[4:]),1))
            gdp['ds'] = pd.to_datetime(gdp['ds'])

            gdp_name = (lambda x: x.split('>', maxsplit=1)[1] if (">" in x) else 'quo')([y for y in gdp.columns if y != 'ds'][0])
            gdp = gdp.rename(columns={[y for y in gdp.columns if y != 'ds'][0] : (lambda x: x.split('>', maxsplit=1)[0] if (">" in x) else x)([y for y in gdp.columns if y != 'ds'][0])})
            regs = gdp.copy()

            macro_scenarios = monthly_fc[monthly_cols]
            macro_scenarios.columns = 'non_dom_' + macro_scenarios.columns
            macro_scenarios = macro_scenarios.rename(columns={'non_dom_ds' : 'ds'})
            macro_scenarios = macro_scenarios.merge(regs, how='left', on='ds')

            # Non_domestic:
            macro_scenarios['delta_gdp'] =  (macro_scenarios['gdp'] - macro_scenarios['non_dom_nominal_gdp']) * gdp_regressor.coef_[0][0]
            macro_scenarios['non_dom_multiplicative_terms'] = macro_scenarios['non_dom_multiplicative_terms'] + macro_scenarios['delta_gdp']
            macro_scenarios['non_dom_forecast'] = macro_scenarios['non_dom_trend'] * (1 + macro_scenarios['non_dom_multiplicative_terms'])

            macro_scenarios_names.append(gdp_name)

            if counter == 0:
                  macro_scenarios = macro_scenarios[['ds', 'non_dom_forecast']]
                  macro_scenarios.loc[:,'gdp_scenario'] = gdp_name
            else:
                  aux = macro_scenarios[['ds', 'non_dom_forecast']]
                  aux.loc[:,'gdp_scenario'] = gdp_name
                  macro_scenarios = pd.concat([macro_scenarios, aux], axis=0)
            
            counter = counter + 1
      
      return macro_scenarios



In [52]:
generate_macro_scenarios(monthly_fc, [gdp_dict]).sort_values('ds')

,ds,non_dom_forecast,gdp_scenario
0,2015-01-01,17670.937655,quo
1,2015-02-01,15940.323470,quo
2,2015-03-01,17084.524622,quo
3,2015-04-01,15429.285843,quo
4,2015-05-01,15269.103116,quo
...,...,...,...
109,2024-02-01,14082.162973,quo
110,2024-03-01,14443.814170,quo
111,2024-04-01,13151.175324,quo
112,2024-05-01,13157.965672,quo


In [ ]:
# This has to be corrected 


def generate_weather_scenarios(self, weather_scenarios : pd.DataFrame):
  """
  Daily weighting for each weather scenario taking into account BHs and Weather Scenarios.
  """

  if len(self.macro_scenarios_names) == 0:
      raise Exception(
          "run 'generate_macro_scenarios' method first.")

  macro_scenarios = self.macro_scenarios
  self.daily_weightings = {}

  for weather in [x for x in weather_scenarios.columns if x != 'ds']:
    
    weather_df = weather_scenarios[['ds', weather]]
    weather_name = (lambda x: x.split('>', maxsplit=1)[1] if (">" in x) else 'quo')([y for y in weather_df.columns if y != 'ds'][0])
    suffix = '_'+weather_name
    weather_df['hdd'] = self.get_hdd(weather_df[weather].to_numpy())
    weather_df['cdd'] = self.get_cdd(weather_df[weather].to_numpy())
    weather_df = weather_df.rename(columns={weather : weather+suffix,
                                            'hdd' : 'hdd'+suffix,
                                            'cdd' : 'cdd'+suffix})

    self.weather_scenarios_names.append(weather_name)

    # non_dom daily weighting for that weather input:
    non_dom_df = self.full_forecasts['non_dom_d'].copy()
    non_dom_df['date_month'] = non_dom_df['ds'].dt.to_period('M')
    non_dom_df = non_dom_df.merge(weather_df, how='inner', on='ds')
    non_dom_df['temperature_delta'] = (non_dom_df[weather+suffix] - non_dom_df['nominal_temperature']) * self.temperature_regressors['non_dom_d'].coef_[0][0]
    non_dom_df['hdd_delta'] = (non_dom_df['hdd'+suffix] - non_dom_df['nominal_hdd']) * self.hdd_regressors['non_dom_d'].coef_[0][0]
    non_dom_df['cdd_delta'] = (non_dom_df['cdd'+suffix] - non_dom_df['nominal_cdd']) * self.cdd_regressors['non_dom_d'].coef_[0][0]
    non_dom_df['multiplicative_terms'] = non_dom_df['multiplicative_terms'] + non_dom_df['temperature_delta'] + non_dom_df['hdd_delta'] + non_dom_df['cdd_delta']
    non_dom_df['bh_free_detrended'] = ( 1 + non_dom_df['multiplicative_terms'] - non_dom_df['bh_effect'] )
    non_dom_df['bh_free_weighting'] = non_dom_df.groupby(by='date_month')['bh_free_detrended'].transform(lambda x: x/x.sum())
    non_dom_df['daily_weighting'] = non_dom_df['bh_free_weighting'] * (1 + non_dom_df['bh_effect'])
    non_dom_df['date_month'] = pd.to_datetime(non_dom_df['date_month'].astype(str), format='%Y-%m')
    self.daily_weightings['non_dom_d'] = non_dom_df[['ds', 'daily_weighting']].copy()
    self.daily_weightings['non_dom_d'].loc[:,'weather_scenario'] = weather_name

  self.final_scenarios = pd.DataFrame( { 'ds' : pd.date_range(initial_date, self.forecast_date, freq='D')} )
  self.final_scenarios['date_month'] = self.final_scenarios['ds'].dt.to_period('M')
  self.final_scenarios['date_month'] = pd.to_datetime(self.final_scenarios['date_month'].astype(str), format='%Y-%m')

  macro_scenarios_df = self.macro_scenarios.copy()
  macro_scenarios_df = macro_scenarios_df.rename(columns={'ds':'date_month'})
  weather_scenarios_df = self.daily_weightings.copy()

  macro = pd.Series(name='macro_scenario', data=self.macro_scenarios_names)
  weather = pd.Series(name='weather_scenario', data=self.weather_scenarios_names)

  self.final_scenarios = self.final_scenarios.merge(macro, how='cross').merge(weather, how='cross') # all combinations given as an input.
  self.final_scenarios = self.final_scenarios.merge(macro_scenarios_df, how='left', on=['date_month', 'macro_scenario']) # merging with our macro_scenarios
  self.final_scenarios = self.final_scenarios.merge(weather_scenarios_df['non_dom_d'].rename(columns={'daily_weighting' : 'non_dom_daily_weighting'}), how='left', on=['ds', 'weather_scenario']) # merging with our daily weightings depending on the weather scenarios.
  self.final_scenarios['non_dom_forecast'] = self.final_scenarios['non_dom_forecast'] * self.final_scenarios['non_dom_daily_weighting']
  self.final_scenarios = self.final_scenarios[['ds', 'non_dom_forecast', 'macro_scenario', 'weather_scenario',  'gdp_scenario']]

  return self.final_scenarios